#  TorchText

## Load stanford dataset to pandas dataframe df

In [26]:
pip install pytreebank

In [27]:
# Load data
import pytreebank
import sys
import os

out_path = os.path.join(sys.path[0], 'sst_{}.txt')
dataset = pytreebank.load_sst('./raw_data')

# Store train, dev and test in separate files
for category in ['train', 'test', 'dev']:
    with open(out_path.format(category), 'w') as outfile:
        for item in dataset[category]:
            outfile.write("__label__{}\t{}\n".format(
                item.to_labeled_lines()[0][0] + 1,
                item.to_labeled_lines()[0][1]
            ))
# Print the length of the training set
print(len(dataset['train']))

8544


In [28]:
import pandas as pd
# Read train data
df = pd.read_csv('../content/sst_train.txt', sep='\t', header=None, names=['truth', 'text'])
df['truth'] = df['truth'].str.replace('__label__', '')
df['truth'] = df['truth'].astype(int).astype('category')
df.head()

,truth,text
0,4,The Rock is destined to be the 21st Century 's...
1,5,The gorgeously elaborate continuation of `` Th...
2,4,Singer/composer Bryan Adams contributes a slew...
3,3,You 'd think by now America would have had eno...
4,4,Yet the act is still charming here .


In [29]:
df.columns = ['labels', 'tweets']
df.head()

,labels,tweets
0,4,The Rock is destined to be the 21st Century 's...
1,5,The gorgeously elaborate continuation of `` Th...
2,4,Singer/composer Bryan Adams contributes a slew...
3,3,You 'd think by now America would have had eno...
4,4,Yet the act is still charming here .


In [30]:
df.labels.value_counts()

4    2322
2    2218
3    1624
5    1288
1    1092
Name: labels, dtype: int64

In [31]:
##Replace the labels very_negative(1) and negative(2) with 0 ; neutral (3) with 2; positive(4) and very_positive(5) with 1.
## This new 0,1,2 matches the tweets labels that we had originally
df['labels'] = df['labels'].replace([1,2],0)
df['labels'] = df['labels'].replace([3],2)
df['labels'] = df['labels'].replace([4,5],1)
df.labels.value_counts()

1    3610
0    3310
2    1624
Name: labels, dtype: int64

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [32]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [33]:
Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [34]:
fields = [('tweets', Tweet),('labels',Label)]
#fields = [('text', Tweet),('truth',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [35]:
df.columns

Index(['labels', 'tweets'], dtype='object')

In [36]:
example = [data.Example.fromlist([df.tweets[i],df.labels[i]], fields) for i in range(df.shape[0])] 
#example = [data.Example.fromlist([df.text[i],df.truth[i]], fields) for i in range(df.shape[0])] 

In [37]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

twitterDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [38]:
(train_data, valid_data) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [39]:
(len(train_data), len(valid_data))

(7262, 1282)

An example from the dataset:

In [40]:
vars(train_data.examples[11])

{'labels': 0,
 'tweets': ['A',
  'Rumor',
  'of',
  'Angels',
  'does',
  "n't",
  'just',
  'slip',
  '--',
  'it',
  'avalanches',
  'into',
  'forced',
  'fuzziness',
  '.']}

In [41]:
vars(train_data.examples[10])

{'labels': 0,
 'tweets': ['The',
  'film',
  'does',
  "n't",
  'show',
  'enough',
  'of',
  'the',
  'creative',
  'process',
  'or',
  'even',
  'of',
  'what',
  'was',
  'created',
  'for',
  'the',
  'non',
  '-',
  'fan',
  'to',
  'figure',
  'out',
  'what',
  'makes',
  'Wilco',
  'a',
  'big',
  'deal',
  '.']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [42]:
Tweet.build_vocab(train_data)
Label.build_vocab(train_data)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [43]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  15741
Size of label vocab :  3
Top 10 words appreared repeatedly : [('.', 6832), (',', 6060), ('the', 5189), ('of', 3767), ('and', 3756), ('a', 3755), ('to', 2588), ('-', 2342), ("'s", 2158), ('is', 2151)]
Labels :  defaultdict(<function _default_unk_index at 0x7f85e3077378>, {1: 0, 0: 1, 2: 2})


In [84]:
Tweet.vocab.itos[1]

'<pad>'

**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [45]:
train_iterator, valid_iterator = data.BucketIterator.splits((train_data, valid_data), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [46]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [413]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [414]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [415]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(15741, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 4,964,203 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [416]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

### Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [417]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return words
    else:
        remaning_pad_len = len(words) - len(remaining)
        pads = [1]*remaning_pad_len
        remaining = torch.cat([torch.tensor(remaining), torch.tensor(pads)], dim=0)
        return remaining

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [418]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

In [419]:
def random_aug(sentence, dropout=0.4):
  rate = random.random()
  if rate > 0.3:
    n = random.randint(5, 9)
    sentence = random_swap(sentence, n=n)
    sentence = random_deletion(sentence, dropout)
    return sentence
  else:
    return sentence

## **Training Loop**

In [420]:
print(model)

classifier(
  (embedding): Embedding(15741, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)


In [421]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweets, tweet_lengths = batch.tweets
        
        tweets_list = [random_aug(tweet.cpu(), dropout=0.5) for tweet in tweets]
        tweets = torch.stack(tweets_list).long().to(device)

        # convert to 1D tensor
        predictions = model(tweets, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        #print(predictions)
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), tweets

**Evaluation Loop**

In [422]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [423]:
N_EPOCHS = 20
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc, tweet = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    print("EPOCH: ", epoch)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

EPOCH:  0
	Train Loss: 1.059 | Train Acc: 42.28%
	 Val. Loss: 1.059 |  Val. Acc: 42.15% 

EPOCH:  1
	Train Loss: 1.052 | Train Acc: 41.62%
	 Val. Loss: 1.056 |  Val. Acc: 41.54% 

EPOCH:  2
	Train Loss: 1.048 | Train Acc: 43.79%
	 Val. Loss: 1.045 |  Val. Acc: 46.11% 

EPOCH:  3
	Train Loss: 1.037 | Train Acc: 46.97%
	 Val. Loss: 1.015 |  Val. Acc: 52.21% 

EPOCH:  4
	Train Loss: 1.004 | Train Acc: 52.23%
	 Val. Loss: 0.982 |  Val. Acc: 55.41% 

EPOCH:  5
	Train Loss: 0.985 | Train Acc: 55.01%
	 Val. Loss: 0.970 |  Val. Acc: 56.25% 

EPOCH:  6
	Train Loss: 0.956 | Train Acc: 58.57%
	 Val. Loss: 0.967 |  Val. Acc: 58.38% 

EPOCH:  7
	Train Loss: 0.947 | Train Acc: 59.09%
	 Val. Loss: 0.978 |  Val. Acc: 55.79% 

EPOCH:  8
	Train Loss: 0.932 | Train Acc: 61.10%
	 Val. Loss: 0.963 |  Val. Acc: 57.24% 

EPOCH:  9
	Train Loss: 0.919 | Train Acc: 62.18%
	 Val. Loss: 0.955 |  Val. Acc: 57.09% 

EPOCH:  10
	Train Loss: 0.908 | Train Acc: 63.47%
	 Val. Loss: 0.947 |  Val. Acc: 59.07% 

EPOCH:  1

torch.Tensor

## Model Testing

In [424]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [425]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'Positive'